In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Add padding token

# Function to train a model
def train_model(model_name, dataset_path, output_dir, epochs=1):
    try:
        # Load the pre-trained model
        model = GPT2LMHeadModel.from_pretrained(model_name)

        # Prepare the dataset using Datasets library
        dataset = load_dataset('text', data_files=dataset_path)['train']
        dataset = dataset.map(lambda examples: tokenizer(examples['text'], truncation=True, padding=True, max_length=128), batched=True)
        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

        # Data collator
        data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

        # Training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=epochs,
            per_device_train_batch_size=4,
            save_steps=10_000,
            save_total_limit=2,
            logging_dir=f"{output_dir}/logs",
            logging_steps=500,
            report_to="none"  # Disable logging for simplicity
        )

        # Trainer initialization
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset,
            data_collator=data_collator,
        )

        # Train the model
        trainer.train()
        trainer.save_model(output_dir)

        return output_dir
    except Exception as e:
        print(f"Error in training {model_name}: {str(e)}")
        raise

# Train models and store paths
trained_models = {}
try:
    print("Training GPT-2...")
    trained_models['GPT-2'] = train_model('gpt2', 'sherlock.txt', './fine_tuned_gpt2', epochs=1)
    print("Training DistilGPT-2...")
    trained_models['DistilGPT-2'] = train_model('distilgpt2', 'sherlock.txt', './fine_tuned_distilgpt2', epochs=1)
except Exception as e:
    print(f"Error during training: {str(e)}")

# Load fine-tuned models if training succeeded
models = {}
for model_name, path in trained_models.items():
    try:
        models[model_name] = pipeline('text-generation', model=path, tokenizer=tokenizer, truncation=True, pad_token_id=50256)
    except Exception as e:
        print(f"Error loading model {model_name}: {str(e)}")

# Function to generate suggestions
def get_model_suggestions(sentence_a, num_words=3):
    suggestions = {}
    for model_name, generator in models.items():
        try:
            # Generate next word suggestions
            result = generator(
                sentence_a,
                max_new_tokens=1,  # Generate only the next word
                num_return_sequences=num_words,  # Generate multiple suggestions
                temperature=0.7,  # Control randomness
                top_p=0.9,  # Nucleus sampling
                top_k=50,  # Limit token set
                do_sample=True  # Enable sampling
            )
            suggestions[model_name] = [r['generated_text'][len(sentence_a):].strip().split()[0] for r in result]

            # Remove duplicates while maintaining order
            suggestions[model_name] = list(dict.fromkeys(suggestions[model_name]))
        except Exception as e:
            suggestions[model_name] = [f"Error: {str(e)}"]
    return suggestions



Training GPT-2...


Step,Training Loss
500,3.961700
1000,3.771300
1500,3.736400
2000,3.670700
2500,3.613100
3000,3.561800


Training DistilGPT-2...


Map:   0%|          | 0/12310 [00:00<?, ? examples/s]

Step,Training Loss
500,4.155700
1000,3.943400
1500,3.937900
2000,3.853800
2500,3.811400
3000,3.762600


In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        if not data or 'sentence_a' not in data:
            return jsonify({'error': 'Invalid input: sentence_a missing'}), 400

        sentence_a = data['sentence_a']
        num_words = data.get('num_words', 3)
        if not isinstance(num_words, int) or num_words < 1 or num_words > 10:
            return jsonify({'error': 'num_words must be an integer between 1 and 10'}), 400

        # Get next-word suggestions from models
        suggestions = get_model_suggestions(sentence_a, num_words)

        print("Suggestions returned to frontend:", suggestions)  # Debugging print

        return jsonify({'suggestions': suggestions})
    except Exception as e:
        print("Error in prediction:", str(e))  # Debugging print
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.24.174:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2024 15:28:54] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:54] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:54] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:58] "POST /predict HTTP/1.1" 200 -


Suggestions returned to frontend: {'GPT-2': ['it', 'not', 'the'], 'DistilGPT-2': ['not', 'you']}
Suggestions returned to frontend: {'GPT-2': ['ress', 'ain', '.'], 'DistilGPT-2': ['ear', '�', 'uc']}
Suggestions returned to frontend: {'GPT-2': ['not', 'that', 'it'], 'DistilGPT-2': [',', '.', 'ing']}


127.0.0.1 - - [28/Nov/2024 15:28:59] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:59] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:28:59] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:29:00] "POST /predict HTTP/1.1" 200 -


Suggestions returned to frontend: {'GPT-2': ['onder', "'"], 'DistilGPT-2': ['�']}
Suggestions returned to frontend: {'GPT-2': ['?', '.'], 'DistilGPT-2': ['-', '!']}


...

Suggestions returned to frontend: {'GPT-2': ['�'], 'DistilGPT-2': ['�', 'I']}
Suggestions returned to frontend: {'GPT-2': ['enough', 'at'], 'DistilGPT-2': ['-', 'to']}
Suggestions returned to frontend: {'GPT-2': ['�'], 'DistilGPT-2': ['�', 'I']}
Suggestions returned to frontend: {'GPT-2': ['in', 'bs'], 'DistilGPT-2': ['to', 'out']}


127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -


Suggestions returned to frontend: {'GPT-2': ['zy', '-', 'aming'], 'DistilGPT-2': [',', '!']}
Suggestions returned to frontend: {'GPT-2': ['rieved'], 'DistilGPT-2': ['rieved']}
Suggestions returned to frontend: {'GPT-2': ['afraid', 'a', 'sure'], 'DistilGPT-2': ['so', 'glad']}


127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 16:01:47] "POST /predict HTTP/1.1" 200 -


Suggestions returned to frontend: {'GPT-2': ['afraid', 'of'], 'DistilGPT-2': ['indebted', 'sure', 'going']}
Suggestions returned to frontend: {'GPT-2': ['little', 'man', 'few'], 'DistilGPT-2': ['man', 'rather', 'long']}
Suggestions returned to frontend: {'GPT-2': ['was', 'had'], 'DistilGPT-2': ['have', 'did', 'found']}
Suggestions returned to frontend: {'GPT-2': ['had', 'could', 'was'], 'DistilGPT-2': ['came', 'was', 'had']}
